# Question 3 
Using the Coffee stack exchange data and posts (both question and answers),
implement a simple inverted index with counts retrieval system (Convert all the text to
lowercase, remove stop words and punctuations). To find tokens, use the NLTK tool, with word
tokenizer. Use term-at-a-time with document count as similarity score 

# Section 1
Modules:<br/>
>removeHTML(strToEvaluate)<br/>
removeNonChar(strToEvaluate)<br/>
processPosts(body) <br/>
iterateThroughWords(filtered_sentence,queryWordMatrixDict,record_id)



In [1]:
# - - - - - - - - - - - - - - - - - - - - - - - - 
# Section 1 of 5 - Execute First to load in memory
# - - - - - - - - - - - - - - - - - - - - - - - - 

# - - - - - - - - - - - - - - - - - - - - - - - - 
# Remove HTML and replace with empty strings
# - - - - - - - - - - - - - - - - - - - - - - - - 

import re

def removeHTML(strToEvaluate):
  """
  This function takes in a string and returns the string 
  cleared of any html.
  ARGS:
    @strToEvaluate:     the original string to be cleared
  RETURNS:
    @result: the resulting string without html tags
  """

  result = re.sub(r'<.*?>', '', strToEvaluate)
  return result

# - - - - - - - - - - - - - - - - - - - - - - - - 
# remove anything that isn't a letter
# - - - - - - - - - - - - - - - - - - - - - - - - 

def removeNonChar(strToEvaluate):
  """
  This function takes in a string and returns the string 
  cleared of any character that isn't A-Z or a-z.
  ARGS:
    @strToEvaluate:     the original string to be cleared
  RETURNS:
    @result: the resulting string reduced to A-Z or a-z only
  """
  
  result = re.sub(r'[^a-zA-Z ]+', '', strToEvaluate)

  return result

# - - - - - - - - - - - - - - - - - - - - - - - - 
# subroutine to process posts
# - - - - - - - - - - - - - - - - - - - - - - - - 

def processPosts(body):
  """
  This function takes in a string and returns the tokenized string 
  that has been cleaned up.
  ARGS:
    @body:     the original string to be cleaned and tokenized
  RETURNS:
    @filtered_sentence: the resulting tokenized string
  """

  # remove html
  stringNoHTML = removeHTML(body)
  
  # remove anything that isn't a character
  stringOnlyChar = removeNonChar(stringNoHTML)

  # convert all to lowercase
  stringLowercase = stringOnlyChar.lower()

  # tokenize the words
  word_tokens = word_tokenize(stringLowercase)

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)
  
  return filtered_sentence

# - - - - - - - - - - - - - - - - - - - - - - - - 
# Iterate through each word in each record
# - - - - - - - - - - - - - - - - - - - - - - - - 

def iterateThroughWords(filtered_sentence,queryWordMatrixDict,record_id):
  """
  This function takes in a tokenized record, the parent dictionary and 
  the ID of the record. It checks to see if the word is already in the 
  dictionary. If not, it adds it with a count of 1. If it is already there
  it checks to see if the ID is already listed. If it is, it adds 1 to the 
  count. If not, it adds the ID with a count of 1.
  ARGS:
    @filtered_sentence:    the set of words for this record
    @queryWordMatrixDict:  the parent dictionary that holds the words, the IDs
                           and the counts
    @record_id:            The ID of the record that is being evaluated
  RETURNS:
    @queryWordMatrixDict:  the parent dictionary that holds the words, the IDs
                           and the counts after the words, IDs, and counts have
                           been added
  """

  for specificWord in filtered_sentence:

    if specificWord in queryWordMatrixDict:
      # the word is already in the parent dictionary
      # check if the record_id is already there

      if record_id in queryWordMatrixDict[specificWord]:
        # word and ID already there. Add one to the count
        queryWordMatrixDict[specificWord][record_id] = queryWordMatrixDict[specificWord][record_id] + 1
      else:
        # word is there but not this question ID yet
        queryWordMatrixDict[specificWord][record_id] = 1

    else:
      # word is not in the parent dictionary, add it
      queryWordMatrixDict[specificWord] = {}
      queryWordMatrixDict[specificWord][record_id] = 1

  return queryWordMatrixDict


# Section 2
Modules:<br/>
>createInvertedIndex()<br/>



In [2]:
# - - - - - - - - - - - - - - - - - - - - - - - - 
# Section 2 of 5 - Execute second to load into memory
# - - - - - - - - - - - - - - - - - - - - - - - - 

# Read in the posts and create an inverted index with counts
# first load all the modules


import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# load stop words for later use
stop_words = set(stopwords.words('english'))

def createInvertedIndex():
  """
  This function tokenizes the data file. It takes no arguments and 
  returns the inverted index. 
  RETURNS:
    @queryWordMatrixDict:  the parent dictionary that holds the words, the IDs
                           and the counts after the words, IDs, and counts have
                           been added
  """

  from post_parser_record import PostParserRecord
  post_reader = PostParserRecord("Posts_Coffee.xml")

  # define the parent dictionary for the words
  queryWordMatrixDict = {}

  # - - - - - - - - - - - - - - - - - - - -
  # iterate through each question post
  # - - - - - - - - - - - - - - - - - - - -

  for question_id in post_reader.map_questions:
    question = post_reader.map_questions[question_id]

    # concatenate title and body
    fullQuestion = question.title + " " + question.body

    filtered_sentence = processPosts(fullQuestion)

    # iterate through each word in this question
    queryWordMatrixDict = iterateThroughWords(filtered_sentence,queryWordMatrixDict,question_id)

  # - - - - - - - - - - - - - - - - - - - -
  # iterate through each answer post
  # - - - - - - - - - - - - - - - - - - - -

  for answer_id in post_reader.map_just_answers:

    answer = post_reader.map_just_answers[answer_id]
    answerBody = answer.body 

    filtered_sentence = processPosts(answerBody)

    # iterate through each word in this answer
    queryWordMatrixDict = iterateThroughWords(filtered_sentence,queryWordMatrixDict,answer_id)

  # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
  # the dictionary is fully populated now with words, IDs and counts
  # sort each word entry by count, in reverse
  # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

  for id, counts in queryWordMatrixDict.items():
    queryWordMatrixDict[id] = sorted(queryWordMatrixDict[id].items(), key=lambda x: x[1], reverse=True)
    
  # print(queryWordMatrixDict)

  return queryWordMatrixDict


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Section 3
Modules:<br/>
>no_compare_top_k(queryWord, top_k_num, parentDict)<br/>
compare_words_AND(words,top_k_num,parentDict)<br/>
compare_words_OR(words,top_k_num,parentDict)



In [3]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# Section 3 of 5 - Execute third to load into memory
# - - - - - - - - - - - - - - - - - - - - - - - - - - -

import sys

# - - - - - - - - - - - - - - - - - - - - - - - - - - -
# print the top k IDs from inverted index
# - - - - - - - - - - - - - - - - - - - - - - - - - - -

def no_compare_top_k(queryWord, top_k_num, parentDict):
  """
  This function takes in a single query word and checks the inverted index 
  for the word referenced. If it is found, it prints out the top K results. 
  Nothing is returned. 
  ARGS:
    @queryWord:    the word for the query
    @top_k_num:    the number of terms to be printed
    @parentDict:   The inverted index to be queried upon
  
  """
  # select the top k results from the queryWord
  if not parentDict[queryWord]:
    # that term isn't there
    print("There are no results")

  count = 0

  if parentDict[queryWord]:
    print("Top",top_k_num," ",queryWord)
    print("Counter\trecordID, number of times word in record")
    for count in range(top_k_num):
      print(count,"\t",parentDict[queryWord][count])
      count+=1

# - - - - - - - - - - - - - - - - - - - - - - - - - - -
# compare >1 words with AND print the top k IDs
# - - - - - - - - - - - - - - - - - - - - - - - - - - -

def compare_words_AND(words,top_k_num,parentDict):
  """
  This function takes in 2 or more query word and compares the results to 
  come up with the set of IDs and values that match. If there are any results, 
  it prints out the top K results. 
  Nothing is returned. 
  ARGS:
    @words:        the words for the query
    @top_k_num:    the number of terms to be printed
    @parentDict:   The inverted index to be queried upon
  
  """

  results = []

  for term in words:
    if term not in parentDict:
      print("There are no results for this")
      sys.exit()

    partialResults = parentDict[term]

    if len(results) == 0:
      results = list(set(partialResults))
    else:
      results = list(set(results) & set(partialResults))

  count = 0

  print("Top ",top_k_num," AND for ", end='')
  for term in words:
    print(term,",", end='')
  print()
  for count in range(top_k_num):
    print(count, "\t", results[count])
    count+=1


# - - - - - - - - - - - - - - - - - - - - - - - - - - -
# compare >1 words with OR and print top k IDs
# - - - - - - - - - - - - - - - - - - - - - - - - - - -

def compare_words_OR(words,top_k_num,parentDict):
  """
  This function takes in 2 or more query word and compares the results to 
  come up with the set of IDs and values that do not match. If there are any 
  results, it prints out the top K results. 
  Nothing is returned. 
  ARGS:
    @words:        the words for the query
    @top_k_num:    the number of terms to be printed
    @parentDict:   The inverted index to be queried upon
  
  """
  results = []

  for term in words:
    if term not in parentDict:
      continue

    partialResults = parentDict[term]

    if len(results) == 0:
      results = list(set(partialResults))
    else:
      results = list(set(results + partialResults))

  count = 0

  print("Top ",top_k_num," OR for ", end='')
  for term in words:
    print(term,",", end='')
  print()
  for count in range(top_k_num):
    print(count, "\t", results[count])
    count+=1


# Section 4
Modules:<br/>
>exportToTSV(parentDict)



In [27]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# Section 4 of 5 - Execute fourth to load into memory
# - - - - - - - - - - - - - - - - - - - - - - - - - - -

# - - - - - - - - - - - - - - - - - - - - - - - - - - -
# export the inverted index to a TSV file
# - - - - - - - - - - - - - - - - - - - - - - - - - - -
import csv 
import collections

from collections import OrderedDict

def exportToTSV(parentDict):
  """
  This function takes the inverted index and prints to a TSV file. 
  Nothing is returned. 
  ARGS:
    @parentDict:   the inverted index 
  """
  # make a new dict from the sorted dict
  od = collections.OrderedDict(sorted(parentDict.items()))
  
  with open('output.tsv', 'w', newline='') as f_output:
    #tsv_output = csv.writer(f_output, delimiter='\t')
    for word in od.keys():
      f_output.write("%s"%(word))
      for k,v in od[word]:
        key = str(k)
        value = str(v)
        f_output.write("\t%s:%s"%(key,value))

      f_output.write("\n")
  


# Section 5
Main program<br/>
>update for any changes to the parameters being sent, then execute the section after executing sections 1 - 4

In [28]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Main Section (5 of 5) - Make any changes for words to compare, 
# and number for k.
# Execute last to run the program
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def main():
  """
  This is the main method. Nothing is taken in and nothing is returned. 
  """
  
  # read in data and create the inverted index
  parentDict = createInvertedIndex()

  # once index complete, execute the queries
  no_compare_top_k("espresso",10,parentDict)

  words_to_compare = ["coffee","turkish"]
  compare_words_AND(words_to_compare,10,parentDict)

  words_to_compare = ["coffee","persian"]
  compare_words_OR(words_to_compare,10,parentDict)

  exportToTSV(parentDict)

main()

Top 10   espresso
Counter	recordID, number of times word in record
0 	 (3269, 30)
1 	 (1574, 20)
2 	 (2095, 16)
3 	 (2077, 13)
4 	 (5537, 13)
5 	 (283, 12)
6 	 (2087, 12)
7 	 (2116, 12)
8 	 (3721, 11)
9 	 (3438, 10)
Top  10  AND for coffee ,turkish ,
0 	 (483, 2)
1 	 (4353, 3)
2 	 (5182, 1)
3 	 (4355, 3)
4 	 (4487, 1)
5 	 (4680, 1)
6 	 (4437, 2)
7 	 (2522, 3)
8 	 (5602, 1)
9 	 (4431, 1)
Top  10  OR for coffee ,persian ,
0 	 (4446, 1)
1 	 (5001, 1)
2 	 (3485, 9)
3 	 (43, 3)
4 	 (2575, 1)
5 	 (5680, 1)
6 	 (46, 12)
7 	 (5276, 1)
8 	 (1561, 1)
9 	 (5112, 1)
